# Cloud Cognitive Environments
Aplicações com modelos LLM

## Requerimentos

* Langchain
* OpenAI
* Pandas
* Python Display
* DocArray
* Wikipedia
* XML to Dict

In [ ]:
# Apagar pasta do repositório
# Faça somente isso se não tenha clonado antes
!rm -rf fiap-ds-cloud-cognitive-environments

In [1]:
# Clonar o repositório da aula
!git clone https://github.com/michelpf/fiap-ds-cloud-cognitive-environments

Cloning into 'fiap-ds-cloud-cognitive-environments'...
remote: Enumerating objects: 241, done.
remote: Counting objects: 100% (69/69), done.
remote: Compressing objects: 100% (53/53), done.
remote: Total 241 (delta 26), reused 52 (delta 16), pack-reused 172
Receiving objects: 100% (241/241), 186.80 MiB | 12.93 MiB/s, done.
Resolving deltas: 100% (77/77), done.
Updating files: 100% (62/62), done.


In [85]:
%cd fiap-ds-cloud-cognitive-environments/aula-4-langchain/

/content/fiap-ds-cloud-cognitive-environments/aula-4-langchain


In [68]:
!pip install openai langchain-community langchain-core langchain tiktoken docarray wikipedia xmltodict langchain-openai chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 5.5 MB/s eta 0:00:00


In [4]:
import os
from openai import OpenAI
import langchain
from IPython.display import display, Markdown as display_markdown
import json
import pandas as pd

# Modelos de base

## OpenAI

Inscreva-se no [site](www.openai.com) para obter uma chave de API. No registro você terá 30 dias de uso gratuito.
Para experimentações pequenas o custo é baixo. Avalie com cuidado ao lançar uma aplicação em produção para incluir limites máximos de gastos.

In [5]:
os.environ["OPENAI_API_KEY"] = "chave"

Utilizaremos o modelo fundacional do ChatGPT 3.5 é ```gpt-3.5-turbo``` conforme [documentação](https://platform.openai.com/docs/models/gpt-4). Sempre revise a documentação pois os modelos podem se tornar obsoletos para a entrada de outros mais novos.

## Chat Completion

Permite criar prompts que conversam com o modelo selecionado. Por meio de roles definidas, podemos criar um prompt mais completo, especificando:

1. ```System```: papel de sistema, ajuda na formatação do tipo de resposta e de como o modelo deve se comportar. Definimos com esse papel aspectos de entrada e saída.
2. ```User```: papel de usuário, neste caso é a informação desejada pelo usuário, que vai seguir as regras definidas no papel anteriormente definido.
2. ```Assistant```: papel de assistente virtual (próprio modelo). Neste caso serve para definir um aspecto de memória.

In [6]:
client = OpenAI(
    api_key=os.environ.get("OPENAI_API_KEY"),
)

In [7]:
model = "gpt-3.5-turbo"

In [8]:
messages=[
        {"role": "system", "content": "Você é um professor de visão computacional que leciona para crianças de até 10 anos."},
        {"role": "user", "content": "Como funciona um detector de bordas de Canny, de forma bem resumida?"},
        {"role": "system", "content": "O resultado precisa ser formatado como Markdown."}
    ]

In [9]:
response = client.chat.completions.create(
    model=model,
    messages=messages,
    temperature=0,
)
response

ChatCompletion(id='chatcmpl-9d4ew6i1bxtf6pW3nIEGiUia2Foog', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Um detector de bordas de Canny funciona em 5 passos principais:\n\n1. **Suavização da imagem**: Reduz o ruído na imagem aplicando um filtro gaussiano.\n2. **Cálculo do gradiente**: Calcula a intensidade do gradiente da imagem para encontrar a direção das mudanças de intensidade.\n3. **Supressão de não-máximos**: Identifica os pixels que representam bordas reais.\n4. **Limiarização dupla**: Define dois limiares para determinar quais bordas são fortes e quais são fracas.\n5. **Rastreamento de bordas por histerese**: Finaliza a detecção de bordas, conectando pixels fortes e eliminando pixels fracos que não estão conectados a bordas fortes.', role='assistant', function_call=None, tool_calls=None))], created=1719099730, model='gpt-3.5-turbo-0125', object='chat.completion', service_tier=None, system_fingerprint=None, usage=Completion

In [10]:
response.choices[0].message.content

'Um detector de bordas de Canny funciona em 5 passos principais:\n\n1. **Suavização da imagem**: Reduz o ruído na imagem aplicando um filtro gaussiano.\n2. **Cálculo do gradiente**: Calcula a intensidade do gradiente da imagem para encontrar a direção das mudanças de intensidade.\n3. **Supressão de não-máximos**: Identifica os pixels que representam bordas reais.\n4. **Limiarização dupla**: Define dois limiares para determinar quais bordas são fortes e quais são fracas.\n5. **Rastreamento de bordas por histerese**: Finaliza a detecção de bordas, conectando pixels fortes e eliminando pixels fracos que não estão conectados a bordas fortes.'

In [11]:
display_markdown(response.choices[0].message.content)

Um detector de bordas de Canny funciona em 5 passos principais:

1. **Suavização da imagem**: Reduz o ruído na imagem aplicando um filtro gaussiano.
2. **Cálculo do gradiente**: Calcula a intensidade do gradiente da imagem para encontrar a direção das mudanças de intensidade.
3. **Supressão de não-máximos**: Identifica os pixels que representam bordas reais.
4. **Limiarização dupla**: Define dois limiares para determinar quais bordas são fortes e quais são fracas.
5. **Rastreamento de bordas por histerese**: Finaliza a detecção de bordas, conectando pixels fortes e eliminando pixels fracos que não estão conectados a bordas fortes.

In [12]:
messages=[
        {"role": "system", "content": "Você é um sistema que extrai 2 informações de um texto, primeiro se a frase for um elogio deverá armazenar is_compliment=True"},
        {"role": "system", "content": "Indique qual o elogio em uma variável compliment, por exemplo em 'A pessoa é bonita', is_compliment=True e compliment='bonita'. "},
        {"role": "user", "content": "Os carros elétricos são incríveis."},
        {"role": "system", "content": "O resultado precisa ser formatado como JSON."}
    ]

In [13]:
response = client.chat.completions.create(
    model=model,
    messages=messages,
    temperature=0,
)
response

ChatCompletion(id='chatcmpl-9d4f9swB7exB8Ofb6NyO3MR43rKIl', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='{\n    "is_compliment": true,\n    "compliment": "incríveis"\n}', role='assistant', function_call=None, tool_calls=None))], created=1719099743, model='gpt-3.5-turbo-0125', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=21, prompt_tokens=106, total_tokens=127))

In [14]:
response.choices[0].message.content

'{\n    "is_compliment": true,\n    "compliment": "incríveis"\n}'

In [15]:
result = json.loads(response.choices[0].message.content)
result

{'is_compliment': True, 'compliment': 'incríveis'}

# Langchain

É um framework com foco em integrar diferentes _foundationals model_, dentre eles o GPT (da OpenAI), Bard (do Google), WatsonX (da IBM) e outros mais. cada modelo e plataformas tem algumas particularidades, mas há um bom espaço para abstrações. É nesse nicho que o framework faz o seu papel de utilizar componentes e serviços comum a todos os modelos como encadeamento de ações, memória, acesso e dados externos e utilização de agentes.

In [16]:
from langchain_openai import ChatOpenAI

## Templates de prompt

Templates ajudam a criar uma forma padronizada sempre que for utilizado um determindo prompt para atividades repetitivas. Por exemplo, traduções, resumos, modificações de tom, etc.

Utilizaremos nos templates parâmetros que são inseridos dentro dos prompts que trazem os dados que são dinâmicos, enquanto parte do prompt continua constante.

A _temperatura_ do chat é o grau de criatividade dado a conversão. Quanto menor, menor menos criativo e randômico ele será, em tese diminui as possibilidades de alucinações, embora os resultados podem ficar mais simples.

Vamos manter zero para extrair o máximo de informação útil, com o mínimo de ruídos.

In [17]:
openai_chat = ChatOpenAI(temperature=0.0, model=model)
openai_chat

ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x79719ac3bb80>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x79719ac552d0>, temperature=0.0, openai_api_key=SecretStr('**********'), openai_proxy='')

Vamos criar prompts para analisar comentários de clientes de modo a toda avaliação seja excluído palavras de ofensa.

In [18]:
style = """
Tom calmo, sintético e respeitoso, sem incluir nenhuma ofensa ou algo parecido.
"""

In [19]:
template_string = """
Analise a revisão de cliente para que sejam resumidas em no máximo 300 caracteres.
Utilize o seguinte estilo {style}.
A revisão é a seguinte: {customer_review}
"""

In [20]:
from langchain.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_template(template_string)

In [21]:
prompt_template.messages[0].prompt

PromptTemplate(input_variables=['customer_review', 'style'], template='\nAnalise a revisão de cliente para que sejam resumidas em no máximo 300 caracteres.\nUtilize o seguinte estilo {style}.\nA revisão é a seguinte: {customer_review}\n')

In [22]:
prompt_template.messages[0].prompt.input_variables

['customer_review', 'style']

In [23]:
customer_review = """
Comprei um brinquedo que não presta! Utilizei todas as instruções
mas não funcionada nada! Que droga!
Se soubesse que fosse assim não teria comprado nessa loja ruim!!!
"""

In [24]:
customer_messages = prompt_template.format_messages(
                    style=style,
                    customer_review=customer_review)

Analisando o resultado com a avaliação de exemplo.

In [25]:
customer_messages

[HumanMessage(content='\nAnalise a revisão de cliente para que sejam resumidas em no máximo 300 caracteres.\nUtilize o seguinte estilo \nTom calmo, sintético e respeitoso, sem incluir nenhuma ofensa ou algo parecido.\n.\nA revisão é a seguinte: \nComprei um brinquedo que não presta! Utilizei todas as instruções\nmas não funcionada nada! Que droga!\nSe soubesse que fosse assim não teria comprado nessa loja ruim!!!\n\n')]

In [26]:
customer_response = openai_chat.invoke(customer_messages)

In [27]:
customer_response.content

'Revisão: O brinquedo comprado não funcionou mesmo seguindo as instruções. Fiquei insatisfeito com a qualidade do produto e não compraria novamente nessa loja.'

Vamos obter uma base de revisões de clientes em sites de comércio eletrônico.
Com esta base vamos sintetizar as reclamações para padronizar análises, removendo palavras e expressões que impactariam muito pouco na avaliação como um todo.

In [28]:
!git clone https://github.com/michelpf/dataset-customer-evaluations

Cloning into 'dataset-customer-evaluations'...
remote: Enumerating objects: 11, done.
remote: Counting objects: 100% (11/11), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 11 (delta 1), reused 7 (delta 0), pack-reused 0
Receiving objects: 100% (11/11), 2.52 MiB | 4.41 MiB/s, done.
Resolving deltas: 100% (1/1), done.


In [29]:
df = pd.read_csv("dataset-customer-evaluations/dataset/am_scrape_final.csv")
df.head()

,Search Query,Product Title,Link,Review
0,smartphone,Smartphone Xiaomi Note 12 4G 128GB 6GB Ram (VE...,https://www.amazon.com.br/dp/B0BZ7RJDHD,Com a necessidade de comprar um celular custo ...
1,smartphone,Smartphone Xiaomi Note 12 4G 128GB 6GB Ram (VE...,https://www.amazon.com.br/dp/B0BZ7RJDHD,Minha experiência de 10 dias de uso com o Xiao...
2,smartphone,Smartphone Xiaomi Note 12 4G 128GB 6GB Ram (VE...,https://www.amazon.com.br/dp/B0BZ7RJDHD,"Smartphone de qualidade como já esperava, boas..."
3,smartphone,Smartphone Xiaomi Note 12 4G 128GB 6GB Ram (VE...,https://www.amazon.com.br/dp/B0BZ7RJDHD,atendeu mto minhas expectativas. Antes eu usa...
4,smartphone,Smartphone Xiaomi Note 12 4G 128GB 6GB Ram (VE...,https://www.amazon.com.br/dp/B0BZ7RJDHD,"Gostei muito do celular, tem resposta rápida e..."


Note que as revisões podem ser extensas, mas não necessáriamente contém informações relevantes em todas as frases.

In [30]:
df.iloc[0]["Review"]

'Com a necessidade de comprar um celular custo benefício comecei pesquisando os modelos que mais vendem no mercado e me deparei com os líderes de sempre: Samsung, Motorola...Apple não é custo-benefício aqui no Brasil.Eu já tive smartphones dessas marcas supracitadas, mas nunca da Xiaomi.Por conseguinte, analisei vários vídeos e tinham varias opções (não cabem ser citadas agora) que entregavam uma boa qualidade de apenas algumas características, porém o conjunto completo deixava sempre a desejar.Partindo da premissa que eu saí de um celular no seguinte estado:-Marca: Samsung-Modelo: Gran prime-Ano de lançamento: 2015-Armazenamento: apenas 8gb de memória interna.Considerando também o valor do mercado atual dos Smartphones: CATASTRÓFICO.Associo que a escolha da marca condiz com o meu objetivo:- Precisava de uma boa tela ( essa tela é a melhor do mercado para esses celulares de entrada).- Tinha a necessidade de uma boa bateria ( essa faz jus à marca, sem contar com o carregamento ultra ráp

In [31]:
customer_messages = prompt_template.format_messages(
                    style=style,
                    customer_review=df.iloc[0]["Review"])

customer_response = openai_chat.invoke(customer_messages)

In [32]:
customer_response.content

'Revisão positiva sobre a Xiaomi, destacando a qualidade da tela, bateria, câmera e desempenho do aparelho. Recomenda a compra sem medo, pois atendeu às necessidades do cliente e superou suas expectativas.'

In [33]:
df.iloc[5]["Review"]

'Gente, é barato e muito bom mesmo; pelo menos até o momento rs. Comprei com 6 g RAM e estou amando, o bicho tá parecendo um foguete de rápido kkkkk.  Estou usando há um mês e meio; o verde é lindo 😍, e uma das coisas que mais me chamou atenção até o momento, é a durabilidade da bateria. Castiguei ele com aplicativos que consomem muita energia e a bateria aguentou 20 horas numa boa, coloquei para carregar pq iria sair e não queria levar o carregador, mas dura muuuuuito, o meu anterior (Zenfone, que tbm é muito bom, a bateria não aguentava 10 horas ou menos). Para carregar, também é mega rápido. E como utilizo o celular na maior parte do tempo para trabalho, consigo visualizar diversos documentos na gigante tela dele rs. Enfim ... até o momento, só elogios para o aparelho, vale a pena comprar.Obs* apenas demora par reiniciar, se estiver com pressa, ferrou! Mas considero isso um inconveniente bobo rs.'

Passando no prompt para resumir e também remover eventuais expressões e agressões.

In [34]:
customer_messages = prompt_template.format_messages(
                    style=style,
                    customer_review=df.iloc[5]["Review"])

customer_response = openai_chat.invoke(customer_messages)
customer_response.content

'Excelente custo-benefício, desempenho rápido e bateria duradoura. Tela grande e carregamento rápido. Único ponto negativo é a demora para reiniciar. Recomendo a compra.'

## Memória e contexto

Adiciona um efeito de contexto sobre as conversas realizadas, tornando a conversa (modelo chat) mais integrado podendo controlar a janela de memória em cada interação.

A memória sempre é passada ao modelo de LLM, portanto existem técnicas que tornam essa passagem mais inteligente, como a limitação de posições, tokens ou até mesmo o uso de sumários.

In [35]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

In [36]:
model = "gpt-3.5-turbo"

Conversação com memória simples.

In [37]:
chat_openai = ChatOpenAI(temperature=0.0, model=model)

memory = ConversationBufferMemory()

conversation = ConversationChain(llm = chat_openai, memory = memory, verbose=True)

In [38]:
conversation.predict(input="Olá, meu nome é Michel e sou o professor na FIAP")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Olá, meu nome é Michel e sou o professor na FIAP
AI:

> Finished chain.


'Olá Michel! É um prazer conhecê-lo. Como posso ajudá-lo hoje?'

In [39]:
conversation.predict(input="Você conhece a FIAP?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Olá, meu nome é Michel e sou o professor na FIAP
AI: Olá Michel! É um prazer conhecê-lo. Como posso ajudá-lo hoje?
Human: Você conhece a FIAP?
AI:

> Finished chain.


'Sim, a FIAP é uma instituição de ensino superior localizada em São Paulo, Brasil. Ela oferece cursos de graduação e pós-graduação nas áreas de tecnologia da informação, administração, engenharia, entre outras. A FIAP também é conhecida por sua forte conexão com o mercado de trabalho e por promover eventos e hackathons para seus alunos.'

In [40]:
conversation.predict(input="Qual é o meu nome e onde eu trabalho?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Olá, meu nome é Michel e sou o professor na FIAP
AI: Olá Michel! É um prazer conhecê-lo. Como posso ajudá-lo hoje?
Human: Você conhece a FIAP?
AI: Sim, a FIAP é uma instituição de ensino superior localizada em São Paulo, Brasil. Ela oferece cursos de graduação e pós-graduação nas áreas de tecnologia da informação, administração, engenharia, entre outras. A FIAP também é conhecida por sua forte conexão com o mercado de trabalho e por promover eventos e hackathons para seus alunos.
Human: Qual é o meu nome e onde eu trabalho?
AI:

> Finished chain.


'Seu nome é Michel e você trabalha como professor na FIAP. Como professor, você provavelmente leciona disciplinas relacionadas à tecnologia da informação, administração ou engenharia, dependendo da sua área de especialização. Como posso ajudá-lo mais hoje, Michel?'

In [41]:
print(memory.buffer)

Human: Olá, meu nome é Michel e sou o professor na FIAP
AI: Olá Michel! É um prazer conhecê-lo. Como posso ajudá-lo hoje?
Human: Você conhece a FIAP?
AI: Sim, a FIAP é uma instituição de ensino superior localizada em São Paulo, Brasil. Ela oferece cursos de graduação e pós-graduação nas áreas de tecnologia da informação, administração, engenharia, entre outras. A FIAP também é conhecida por sua forte conexão com o mercado de trabalho e por promover eventos e hackathons para seus alunos.
Human: Qual é o meu nome e onde eu trabalho?
AI: Seu nome é Michel e você trabalha como professor na FIAP. Como professor, você provavelmente leciona disciplinas relacionadas à tecnologia da informação, administração ou engenharia, dependendo da sua área de especialização. Como posso ajudá-lo mais hoje, Michel?


In [42]:
memory.load_memory_variables({})

{'history': 'Human: Olá, meu nome é Michel e sou o professor na FIAP\nAI: Olá Michel! É um prazer conhecê-lo. Como posso ajudá-lo hoje?\nHuman: Você conhece a FIAP?\nAI: Sim, a FIAP é uma instituição de ensino superior localizada em São Paulo, Brasil. Ela oferece cursos de graduação e pós-graduação nas áreas de tecnologia da informação, administração, engenharia, entre outras. A FIAP também é conhecida por sua forte conexão com o mercado de trabalho e por promover eventos e hackathons para seus alunos.\nHuman: Qual é o meu nome e onde eu trabalho?\nAI: Seu nome é Michel e você trabalha como professor na FIAP. Como professor, você provavelmente leciona disciplinas relacionadas à tecnologia da informação, administração ou engenharia, dependendo da sua área de especialização. Como posso ajudá-lo mais hoje, Michel?'}

In [43]:
memory.save_context({"input": "Eu tenho dois filhos, uma de 10 anos e um de 6 anos."},
                    {"output": "Legal, então você tem um casal."})

In [44]:
memory.load_memory_variables({})

{'history': 'Human: Olá, meu nome é Michel e sou o professor na FIAP\nAI: Olá Michel! É um prazer conhecê-lo. Como posso ajudá-lo hoje?\nHuman: Você conhece a FIAP?\nAI: Sim, a FIAP é uma instituição de ensino superior localizada em São Paulo, Brasil. Ela oferece cursos de graduação e pós-graduação nas áreas de tecnologia da informação, administração, engenharia, entre outras. A FIAP também é conhecida por sua forte conexão com o mercado de trabalho e por promover eventos e hackathons para seus alunos.\nHuman: Qual é o meu nome e onde eu trabalho?\nAI: Seu nome é Michel e você trabalha como professor na FIAP. Como professor, você provavelmente leciona disciplinas relacionadas à tecnologia da informação, administração ou engenharia, dependendo da sua área de especialização. Como posso ajudá-lo mais hoje, Michel?\nHuman: Eu tenho dois filhos, uma de 10 anos e um de 6 anos.\nAI: Legal, então você tem um casal.'}

In [45]:
conversation.predict(input="Em que ano meus filhos nasceram, sabendo que estamos em 2023?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Olá, meu nome é Michel e sou o professor na FIAP
AI: Olá Michel! É um prazer conhecê-lo. Como posso ajudá-lo hoje?
Human: Você conhece a FIAP?
AI: Sim, a FIAP é uma instituição de ensino superior localizada em São Paulo, Brasil. Ela oferece cursos de graduação e pós-graduação nas áreas de tecnologia da informação, administração, engenharia, entre outras. A FIAP também é conhecida por sua forte conexão com o mercado de trabalho e por promover eventos e hackathons para seus alunos.
Human: Qual é o meu nome e onde eu trabalho?
AI: Seu nome é Michel e você trabalha como professor na FIAP. Como professor, você provavelmente leciona disciplinas relacionadas à tecnolog

'Se estamos em 2023, sua filha de 10 anos nasceu em 2013 e seu filho de 6 anos nasceu em 2017. Como posso ajudá-lo mais hoje, Michel?'

In [46]:
from langchain.memory import ConversationBufferWindowMemory

Adicionando limitação de entradas (valor k).

In [47]:
memory_limited = ConversationBufferWindowMemory(k=1)

In [48]:
memory_limited.save_context({"input": "Estudei Engenharia Elétrica na FEI em São Bernardo do Campo."},
                    {"output": "Muito bom! É uma faculdade reconhecida nesta área."})

memory_limited.save_context({"input": "Eu nasci na cidade de São Paulo."},
                    {"output": "Legal!"})

In [49]:
memory_limited.load_memory_variables({})

{'history': 'Human: Eu nasci na cidade de São Paulo.\nAI: Legal!'}

In [50]:
conversation = ConversationChain(llm = chat_openai, memory = memory_limited, verbose=True)

In [51]:
conversation.predict(input="Onde eu me estudei?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Eu nasci na cidade de São Paulo.
AI: Legal!
Human: Onde eu me estudei?
AI:

> Finished chain.


'Desculpe, eu não sei onde você estudou. Posso te ajudar com mais alguma coisa?'

In [52]:
conversation.predict(input="Onde eu nasci?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Onde eu me estudei?
AI: Desculpe, eu não sei onde você estudou. Posso te ajudar com mais alguma coisa?
Human: Onde eu nasci?
AI:

> Finished chain.


'Você nasceu na cidade de São Paulo, Brasil. Posso te ajudar com mais alguma informação?'

In [53]:
from langchain.memory import ConversationTokenBufferMemory

Adicioando limitação por tokens.

In [54]:
memory_limited = ConversationTokenBufferMemory(llm=chat_openai, max_token_limit=10)

In [55]:
memory_limited.save_context({"input": "Estudei Engenharia Elétrica na FEI em São Bernardo do Campo."},
                    {"output": "Muito bom! É uma faculdade reconhecida nesta área."})

memory_limited.save_context({"input": "Eu nasci na cidade de São Paulo."},
                    {"output": "Legal!"})

In [56]:
memory_limited.load_memory_variables({})

{'history': 'AI: Legal!'}

Utilizando sumarização para carregar a memória. Esta funcionalidade utilizada o próprio LLM para gerar o resumo e o carrega como efeito de memória.

In [57]:
from langchain.memory import ConversationSummaryBufferMemory

In [58]:
memory_limited = ConversationSummaryBufferMemory(llm=chat_openai, max_token_limit=100)

In [59]:
memory_limited.save_context(
    {"input": "Estudei Engenharia Elétrica na FEI em São Bernardo do Campo, referência na área e que formou muitos profissionais."},
    {"output": "Muito bom! É uma faculdade reconhecida nesta área."})

memory_limited.save_context(
    {"input": "Eu nasci na cidade de São Paulo, capital do Estado. Esta cidade é uma das maiores do Brasil.\
    Nesta cidadade as pessoas costumam ir ao trabalho utilizando transporte público.\
    O transporte público mais preferido é o metrô, seguido do trem e do ônibus.\
    Muitas pessoas optam por ir de carro, mas as restrições como o rodízio de veículos tornam a necessidade \
    de ter mais de um carro ou ir de transporte público em algum dia da semana."},
    {"output": "Legal!"})

In [60]:
memory_limited.load_memory_variables({})

{'history': 'System: O humano compartilha que estudou Engenharia Elétrica na FEI em São Bernardo do Campo, uma faculdade de referência na área. O AI elogia a faculdade. O humano então fala sobre ter nascido em São Paulo, uma das maiores cidades do Brasil, onde as pessoas costumam usar transporte público para ir ao trabalho, sendo o metrô a opção mais popular, seguido pelo trem e ônibus. Muitas pessoas também optam por ir de carro, mas as restrições como o rodízio de veículos tornam necessário ter mais de um carro ou usar transporte público em algum dia da semana.\nAI: Legal!'}

In [61]:
conversation = ConversationChain(
    llm=chat_openai,
    memory = memory_limited,
    verbose=True
)

In [62]:
conversation.predict(input="Onde eu me formei?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
System: O humano compartilha que estudou Engenharia Elétrica na FEI em São Bernardo do Campo, uma faculdade de referência na área. O AI elogia a faculdade. O humano então fala sobre ter nascido em São Paulo, uma das maiores cidades do Brasil, onde as pessoas costumam usar transporte público para ir ao trabalho, sendo o metrô a opção mais popular, seguido pelo trem e ônibus. Muitas pessoas também optam por ir de carro, mas as restrições como o rodízio de veículos tornam necessário ter mais de um carro ou usar transporte público em algum dia da semana.
AI: Legal!
Human: Onde eu me formei?
AI:

> Finished chain.


'Você se formou na FEI em Engenharia Elétrica. Parabéns pela conquista!'

In [63]:
memory_limited.chat_memory.messages

[AIMessage(content='Legal!'),
 HumanMessage(content='Onde eu me formei?'),
 AIMessage(content='Você se formou na FEI em Engenharia Elétrica. Parabéns pela conquista!')]

##Dados externos com word embeddings

O framework permite criar embeddings a partir de arquivos externos para tornar mais leve a quantidade de dados que é enviado aos LLMs.

Com este técnica, suponha que tenhamos um livro com 1000 páginas, mas somente 1 página é necessária para as análises, os embeddings ajudarão a separar a parte de interesse off-line e enviará somente a página para o modelo de LLM.

Vamos utilizar o mesmo dataset de avaliações de clientes.

In [64]:
!git clone https://github.com/michelpf/dataset-customer-evaluations

fatal: destination path 'dataset-customer-evaluations' already exists and is not an empty directory.


Neste caso, vamos utilizar os dados do Mercado Livre.

In [65]:
df = pd.read_csv("dataset-customer-evaluations/dataset/ml_scrape_final.csv")
df.head()

,Pesquisa,Titulo,Link,Comentario
0,smartphone,Smartphone Samsung Galaxy A14 Dual 6.6 128gb P...,https://produto.mercadolivre.com.br/MLB-331518...,A foto fica amarelada quando eu vou fotografar...
1,smartphone,Smartphone Samsung Galaxy A14 Dual 6.6 128gb P...,https://produto.mercadolivre.com.br/MLB-331518...,👏🏼👏🏼👏🏼👏🏼👏🏼👏🏼.
2,smartphone,Smartphone Samsung Galaxy A14 Dual 6.6 128gb P...,https://produto.mercadolivre.com.br/MLB-331518...,Muito bom.
3,smartphone,Smartphone Samsung Galaxy A14 Dual 6.6 128gb P...,https://produto.mercadolivre.com.br/MLB-331518...,Produto muito bom dei de presente pra meu filh...
4,smartphone,Smartphone Samsung Galaxy A14 Dual 6.6 128gb P...,https://produto.mercadolivre.com.br/MLB-331518...,Recomendo.


Para o arquivo não ficar muito grande, vamos limitar o produto para smartphone.

In [66]:
df.query("Pesquisa == 'smartphone'").to_csv("smartphone_review.csv")

In [75]:
from langchain_openai import ChatOpenAI
from langchain.document_loaders import CSVLoader
from langchain.vectorstores import DocArrayInMemorySearch
from langchain.indexes import VectorstoreIndexCreator
from langchain_openai import OpenAIEmbeddings

Carregamos o arquivo no CSVLoader. Cada tipo de arquivo tem um loader diferente.

In [70]:
loader = CSVLoader(file_path="smartphone_review.csv")

É criado um armazenamento vetorial, que são os embeddings com índices associados para as buscas.

In [71]:
embedding_model = OpenAIEmbeddings(chunk_size=10)
index = VectorstoreIndexCreator(embedding=embedding_model).from_loaders([loader])

/usr/local/lib/python3.10/dist-packages/langchain/indexes/vectorstore.py:129: UserWarning: Using InMemoryVectorStore as the default vectorstore.This memory store won't persist data. You should explicitlyspecify a vectorstore when using VectorstoreIndexCreator
  warnings.warn(


In [72]:
query = "List as 3 piores revisões de clientes, listando com o modelo e o link do produto."

In [76]:
chat_openai = ChatOpenAI(temperature=0.0, model=model)

In [77]:
response = index.query(query, verbose=True, llm=chat_openai)



> Entering new RetrievalQA chain...

> Finished chain.


In [78]:
display_markdown(response)

1. **Smartphone Moto E13 32gb Tela 6.5'' 2gb Ram Grafite Motorola**
   - **Comentário:** O áudio do celular estar ruim e os vídeos sair ruim tbm.
   - **Link:** [Smartphone Moto E13](https://produto.mercadolivre.com.br/MLB-3140442935-smartphone-moto-e13-32gb-tela-65-2gb-ram-grafite-motorola-_JM#position=54&search_layout=stack&type=item&tracking_id=4703b69b-1b0f-4cc2-b281-a641f81b3281)

2. **Smartphone Motorola Moto G22 Dual 6,5 128gb 4gb Ram Azul**
   - **Comentário:** O smartphone é muito lento, realmente se alguém tivesse me dito isso antes não teria comprado.
   - **Link:** [Smartphone Motorola Moto G22](https://produto.mercadolivre.com.br/MLB-2660980219-smartphone-motorola-moto-g22-dual-65-128gb-4gb-ram-azul-_JM#position=48&search_layout=stack&type=item&tracking_id=4703b69b-1b0f-4cc2-b281-a641f81b3281)

3. **Smartphone Samsung Galaxy A14 Dual 6.6 128gb Preto 4gb Ram**
   - **Comentário:** Produto muito bom dei de presente pra meu filho ele tá amando só esperava vir ao menos com o fone de ouvido mais não veio mais do resto perfeito.
   - **Link:** [Smartphone Samsung Galaxy A14](https://produto.mercadolivre.com.br/MLB-3315181641-smartphone-samsung-galaxy-a14-dual-66-128gb-preto-4gb-ram-_JM#position=46&search_layout=stack&type=item&tracking_id=4703b69b-1b0f-4cc2-b281-a641f81b3281)

Também podemos citar as fontes dos documentos utilizados.

In [79]:
response = index.query_with_sources(query, verbose=True, llm=chat_openai)



> Entering new RetrievalQAWithSourcesChain chain...

> Finished chain.


In [80]:
response

{'question': 'List as 3 piores revisões de clientes, listando com o modelo e o link do produto.',
 'answer': "As 3 piores revisões de clientes são:\n1. Modelo: Smartphone Moto E13 32gb Tela 6.5'' 2gb Ram Grafite Motorola\n   Link: https://produto.mercadolivre.com.br/MLB-3140442935-smartphone-moto-e13-32gb-tela-65-2gb-ram-grafite-motorola-_JM#position=54&search_layout=stack&type=item&tracking_id=4703b69b-1b0f-4cc2-b281-a641f81b3281\n   Comentário: O áudio do celular estar ruim e os vídeos sair ruim tbm.\n\n2. Modelo: Smartphone Motorola Moto G22 Dual 6,5 128gb 4gb Ram Azul\n   Link: https://produto.mercadolivre.com.br/MLB-2660980219-smartphone-motorola-moto-g22-dual-65-128gb-4gb-ram-azul-_JM#position=48&search_layout=stack&type=item&tracking_id=4703b69b-1b0f-4cc2-b281-a641f81b3281\n   Comentário: O smartphone é muito lento, realmente se alguém tivesse me dito isso antes não teria comprado.\n\n3. Modelo: Smartphone Samsung Galaxy A14 Dual 6.6 128gb Preto 4gb Ram\n   Link: https://produto

In [82]:
chat_openai = ChatOpenAI(temperature=0.0, model=model)

Adicionamente podemos utilizar um loader de PDF e utilizar outros tipos de indexação offline.

In [83]:
!pip install pypdf faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 29.0 MB/s eta 0:00:00


In [86]:
from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader("docs/SM-A146M_Emb_BR_Rev.1.2.pdf")
pages = loader.load_and_split()

A indexação offline [FAISS](https://python.langchain.com/docs/integrations/vectorstores/faiss) é do Facebook para buscar similiaridade nos textos.

In [87]:
from langchain.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

faiss_index = FAISS.from_documents(pages, OpenAIEmbeddings(chunk_size=10))

In [88]:
query = "O que fazer se o aparelho não liga?"
docs = faiss_index.similarity_search(query)
docs

[Document(page_content='Primeiros passos\n13\nLigar ou desligar seu aparelho\nSiga todos os avisos e instruções recomendadas pelo pessoal autorizado em áreas \nonde aparelhos sem fio são proibidos, tais como aviões e hospitais.\nTecla Lateral\nLigar o aparelho\nMantenha pressionada a Tecla Lateral por alguns segundos para ligar o aparelho.\nDesligar o aparelho\n1 Para desligar o aparelho, mantenha as teclas Lateral e Diminuir volume pressionadas \nsimultaneamente. Como alternativa, abra o painel de notificações e toque em \n .\n2 Toque em Desligar .\nPara reiniciar o aparelho, toque em Reiniciar .\nForçar reinício\nSe o seu aparelho estiver travado e sem operação, mantenha as teclas Lateral e Diminuir volume pressionadas simultaneamente por aproximadamente 7 segundos  \npara reiniciá-lo.\nConfiguração inicial\nAo ligar pela primeira vez ou após executar uma restauração de dados, siga as instruções na tela para configurá-lo.\nSe você não se conectar a uma rede Wi-Fi, você não conseguirá

In [89]:
for doc in docs:
    print(str(doc.metadata["page"]) + ":", doc.page_content)

12: Primeiros passos
13
Ligar ou desligar seu aparelho
Siga todos os avisos e instruções recomendadas pelo pessoal autorizado em áreas 
onde aparelhos sem fio são proibidos, tais como aviões e hospitais.
Tecla Lateral
Ligar o aparelho
Mantenha pressionada a Tecla Lateral por alguns segundos para ligar o aparelho.
Desligar o aparelho
1 Para desligar o aparelho, mantenha as teclas Lateral e Diminuir volume pressionadas 
simultaneamente. Como alternativa, abra o painel de notificações e toque em 
 .
2 Toque em Desligar .
Para reiniciar o aparelho, toque em Reiniciar .
Forçar reinício
Se o seu aparelho estiver travado e sem operação, mantenha as teclas Lateral e Diminuir volume pressionadas simultaneamente por aproximadamente 7 segundos  
para reiniciá-lo.
Configuração inicial
Ao ligar pela primeira vez ou após executar uma restauração de dados, siga as instruções na tela para configurá-lo.
Se você não se conectar a uma rede Wi-Fi, você não conseguirá definir algumas 
funções do aparelho d

Ou podemos fazer a mesma coisa que fizemos com o arquivo CSV e criar os embeddings e depois buscar pelo índice, trazendo o modelo de LLM para ajudar na busca do mesmo.

In [90]:
embedding_model = OpenAIEmbeddings(chunk_size=10)
index = VectorstoreIndexCreator(embedding=embedding_model).from_loaders([loader])

/usr/local/lib/python3.10/dist-packages/langchain/indexes/vectorstore.py:129: UserWarning: Using InMemoryVectorStore as the default vectorstore.This memory store won't persist data. You should explicitlyspecify a vectorstore when using VectorstoreIndexCreator
  warnings.warn(


In [91]:
query = "O que fazer se o aparelho não ligar?"

In [93]:
response = index.query(query, llm=chat_openai)

In [94]:
display_markdown(response)

Se o seu aparelho não ligar, mesmo estando conectado ao carregador, pode ser necessário aguardar alguns minutos para permitir que a bateria se recarregue antes de tentar ligá-lo novamente. Se isso não resolver o problema, você pode tentar reiniciar o aparelho mantendo as teclas Lateral e Diminuir volume pressionadas por mais de 7 segundos. Se ainda assim o problema persistir, pode ser necessário executar uma restauração para o padrão de fábrica. Lembre-se de fazer um backup de todos os dados importantes antes de realizar essa ação.

In [95]:
response = index.query_with_sources(query, llm=chat_openai)

In [96]:
response

{'question': 'O que fazer se o aparelho não ligar?',
 'answer': 'Se o aparelho não ligar, mantenha as teclas Lateral e Diminuir volume pressionadas por mais de 7 segundos para reiniciá-lo. Se o problema persistir, execute uma restauração para o padrão de fábrica. Antes de fazer isso, faça um backup de todos os dados importantes. \n',
 'sources': 'docs/SM-A146M_Emb_BR_Rev.1.2.pdf'}

## Agentes

Os agentes são ações que podem ser disparados a partir dos modelos LLM.

A idéia principal é que cada agente possui um "pensamento", uma determinação de qual "ação" realizar e depois delegar ao "agente" especificado, dentro das descrições passadas.

In [97]:
!pip install langchainhub

In [98]:
from langchain.agents import load_tools
from langchain.agents import AgentType, create_json_chat_agent, create_react_agent, AgentExecutor
from langchain_openai import ChatOpenAI

In [99]:
from langchain import hub

prompt = hub.pull("hwchase17/react-chat-json")

In [100]:
chat_openai = ChatOpenAI(temperature=0, model=model)

Vamos carregar a ferramenta (ou agente) para operações matemáticas.

In [101]:
tools = load_tools(["llm-math"], llm=chat_openai)

In [102]:
agent = create_json_chat_agent(
    chat_openai,
    tools,
    prompt)

In [103]:
agent_executor = AgentExecutor(
    agent=agent, tools=tools, verbose=True, handle_parsing_errors=True
)

In [104]:
agent_executor.invoke({"input": "Quanto é raiz quadrada de 5 menos 0,75?"})



> Entering new AgentExecutor chain...
```json
{
    "action": "Calculator",
    "action_input": "sqrt(5) - 0.75"
}
```Answer: 1.4860679774997898```json
{
    "action": "Final Answer",
    "action_input": "1.4860679774997898"
}
```

> Finished chain.


{'input': 'Quanto é raiz quadrada de 5 menos 0,75?',
 'output': '1.4860679774997898'}

Podemos escolher outra ferramenta, neste caso o Wikipedia, para interagir com os dados armazenados.

In [105]:
tools = load_tools(["wikipedia"], llm=chat_openai)

In [106]:
prompt = hub.pull("hwchase17/react")

In [107]:
agent = create_react_agent(
    chat_openai,
    tools,
    prompt)

In [108]:
agent_executor = AgentExecutor(
    agent=agent, tools=tools, verbose=True, handle_parsing_errors=True
)

In [109]:
agent_executor.invoke({"input": "Quem é o fundador da Gurgel?"})



> Entering new AgentExecutor chain...
I should search for the founder of Gurgel on Wikipedia.
Action: wikipedia
Action Input: Gurgel (empresa)Page: Flávio Rocha
Summary: Flávio Gurgel Rocha (born 14 February 1958) is a Brazilian former federal deputy and businessman, current CEO and Chairman of Lojas Riachuelo, one of the largest retailers in the country.

Page: Automotive industry in Brazil
Summary: The Brazilian automotive industry is coordinated by the Associação Nacional dos Fabricantes de Veículos Automotores (Anfavea), created in 1956, which includes automakers (cars, light vehicles, trucks, buses and agriculture machines) with factories in Brazil. Anfavea is part of the Organisation Internationale des Constructeurs d'Automobiles (OICA), based in Paris. In 2021, the annual production exceeded 2.2 million vehicles, the 8th largest in the world.
Most large global automotive companies are present in Brazil, such as: BMW, BYD, Chery, Fiat, Ford, Geely, General Motors, Honda, Hyunda

{'input': 'Quem é o fundador da Gurgel?',
 'output': 'Flávio Gurgel Rocha is the founder of Gurgel.'}

Existem diversos tipos de ferramentas que podem ser utilizadas, como por exemplo o acesso ao Pubmed, um repositório de artigos médicos.

In [110]:
from langchain.tools import PubmedQueryRun

In [111]:
tool = PubmedQueryRun()

tool.run("Febre Familiar do Mediterrâneo")

'Published: 2017-03-23\nTitle: Clinical outcomes and survival in AA amyloidosis patients.\nCopyright Information: Copyright © 2017. Published by Elsevier Editora Ltda.\nSummary::\nAIM: Amyloid A amyloidosis is a rare complication of chronic inflammatory conditions. Most patients with amyloid A amyloidosis present with nephropathy and it leads to renal failure and death. We studied clinical characteristics and survival in patients with amyloid A amyloidosis.\nMETHODS: A total of 81 patients (51 males, 30 females) with renal biopsy proven amyloid A amyloidosis were analyzed retrospectively. The patients were divided into good and poor outcomes groups according to survival results.\nRESULTS: Most of the patients (55.6%) had nephrotic range proteinuria at diagnosis. Most frequent underlying disorders were familial Mediterranean fever (21.2%) and rheumatoid arthritis (10.6%) in the good outcome group and malignancy (20%) in the poor outcome group. Only diastolic blood pressure in the good o

In [112]:
prompt = hub.pull("hwchase17/react")

In [113]:
agent = create_react_agent(
    chat_openai,
    tools,
    prompt)

In [114]:
agent_executor = AgentExecutor(
    agent=agent, tools=tools, verbose=True, handle_parsing_errors=True
)

In [116]:
agent_executor.invoke({"input": "Busque somente um artigo sobre a Febre Familiar do Mediterrâneo."})



> Entering new AgentExecutor chain...
Você deve usar a ferramenta Wikipedia para encontrar informações específicas sobre a Febre Familiar do Mediterrâneo.
Action: wikipedia
Action Input: Febre Familiar do MediterrâneoPage: Sardinian language
Summary: Sardinian or Sard (endonym: sardu, Sardinian: [ˈsaɾdu], limba sarda, Sardinian: [ˈlimba ˈzaɾda], or lìngua sarda, Sardinian: [ˈliŋɡwa ˈzaɾda]) is a Romance language spoken by the Sardinians on the Western Mediterranean island of Sardinia.
Many Romance linguists consider it, together with Italian, as the language that is the closest to Latin among all Latin's descendants. However, it has also incorporated elements of Pre-Latin (mostly Paleo-Sardinian and, to a much lesser degree, Punic) substratum, as well as a Byzantine Greek, Catalan, Castilian, and Italian superstratum. These elements originate in the political history of Sardinia, whose indigenous society experienced for centuries competition and at times conflict with a series of col

/usr/local/lib/python3.10/dist-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /usr/local/lib/python3.10/dist-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


Page: Familial Mediterranean fever
Summary: Familial Mediterranean fever (FMF) is a hereditary inflammatory disorder.: 149   FMF is an autoinflammatory disease caused by mutations in Mediterranean fever gene, which encodes a 781–amino acid protein called pyrin. While all ethnic groups are susceptible to FMF, it usually occurs in people of Mediterranean origin—including Sephardic Jews, Mizrahi Jews, Ashkenazi Jews, Assyrians, Armenians, Azerbaijanis, Druze, Levantines, Kurds, Greeks, Turks and Italians.
The disorder has been given various names, including familial paroxysmal polyserositis, periodic peritonitis, recurrent polyserositis, benign paroxysmal peritonitis, periodic disease or periodic fever, Reimann periodic disease or Reimann syndrome, Siegal-Cattan-Mamou disease, and Wolff periodic disease. Note that "periodic fever" can also refer to any of the periodic fever syndromes.



Page: Shar Pei
Summary: The Shar Pei (Chinese: 沙皮; Cantonese Yale: shā pèih; pinyin: shā pí) is a dog 

{'input': 'Busque somente um artigo sobre a Febre Familiar do Mediterrâneo.',
 'output': 'A Febre Familiar do Mediterrâneo é uma doença inflamatória hereditária causada por mutações no gene da febre mediterrânea, que codifica uma proteína chamada pirina. A doença geralmente ocorre em pessoas de origem mediterrânea.'}

E por fim, podemos criar nossa própria ferramenta.
O exemplo a seguir será de consumir 2 APIs de testes (mock) que traz uma lista de produtos ou um produto individual, de acordo com um identificador.

In [117]:
import requests
from langchain.agents import tool

Incluímos a anotação "@tool" na função e obrigatóriamente precisa ter entrada e saída em String.

Se não for utiliar a entrada, envie um parâmetro vazio.

Importante é detalhar no comentário da função sobre o que faz e qual parâmetro necessita.

In [118]:
retorno = requests.get("https://dummyjson.com/products")
products = json.loads(retorno.content)
products

{'products': [{'id': 1,
   'title': 'Essence Mascara Lash Princess',
   'description': 'The Essence Mascara Lash Princess is a popular mascara known for its volumizing and lengthening effects. Achieve dramatic lashes with this long-lasting and cruelty-free formula.',
   'category': 'beauty',
   'price': 9.99,
   'discountPercentage': 7.17,
   'rating': 4.94,
   'stock': 5,
   'tags': ['beauty', 'mascara'],
   'brand': 'Essence',
   'sku': 'RCH45Q1A',
   'weight': 2,
   'dimensions': {'width': 23.17, 'height': 14.43, 'depth': 28.01},
   'warrantyInformation': '1 month warranty',
   'shippingInformation': 'Ships in 1 month',
   'availabilityStatus': 'Low Stock',
   'reviews': [{'rating': 2,
     'comment': 'Very unhappy with my purchase!',
     'date': '2024-05-23T08:56:21.618Z',
     'reviewerName': 'John Doe',
     'reviewerEmail': 'john.doe@x.dummyjson.com'},
    {'rating': 2,
     'comment': 'Not as described!',
     'date': '2024-05-23T08:56:21.618Z',
     'reviewerName': 'Nolan Gon

In [119]:
import requests

@tool
def products(text: str) -> str:
    """Retorna lista de produtos de uma loja de
    comércio eletrônico chamada ShopAI.
    Não requer parâmetro de entrada, se necessário informe vazio.
    O resultado será na forma JSON somente com o campo title e price."""

    retorno = requests.get("https://dummyjson.com/products")
    items = json.loads(retorno.content)
    items = items["products"][:3]

    return items

In [120]:
products("")

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `BaseTool.__call__` was deprecated in langchain-core 0.1.47 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


[{'id': 1,
  'title': 'Essence Mascara Lash Princess',
  'description': 'The Essence Mascara Lash Princess is a popular mascara known for its volumizing and lengthening effects. Achieve dramatic lashes with this long-lasting and cruelty-free formula.',
  'category': 'beauty',
  'price': 9.99,
  'discountPercentage': 7.17,
  'rating': 4.94,
  'stock': 5,
  'tags': ['beauty', 'mascara'],
  'brand': 'Essence',
  'sku': 'RCH45Q1A',
  'weight': 2,
  'dimensions': {'width': 23.17, 'height': 14.43, 'depth': 28.01},
  'warrantyInformation': '1 month warranty',
  'shippingInformation': 'Ships in 1 month',
  'availabilityStatus': 'Low Stock',
  'reviews': [{'rating': 2,
    'comment': 'Very unhappy with my purchase!',
    'date': '2024-05-23T08:56:21.618Z',
    'reviewerName': 'John Doe',
    'reviewerEmail': 'john.doe@x.dummyjson.com'},
   {'rating': 2,
    'comment': 'Not as described!',
    'date': '2024-05-23T08:56:21.618Z',
    'reviewerName': 'Nolan Gonzalez',
    'reviewerEmail': 'nolan.g

In [121]:
@tool
def product(text: str) -> str:
    """Retorna o produto baseado na sua identificação.
    Ela é numérica, mas deve ser enviado no formato texto e é obrigatória ser passado.
    Esse é um produto do comércio eletrônico chamada ShopAI.
    O resultado será na forma JSON ou dicionário."""

    retorno = requests.get("https://dummyjson.com/products/"+text)
    item = json.loads(retorno.content)

    return item

In [122]:
product("1")

{'id': 1,
 'title': 'Essence Mascara Lash Princess',
 'description': 'The Essence Mascara Lash Princess is a popular mascara known for its volumizing and lengthening effects. Achieve dramatic lashes with this long-lasting and cruelty-free formula.',
 'category': 'beauty',
 'price': 9.99,
 'discountPercentage': 7.17,
 'rating': 4.94,
 'stock': 5,
 'tags': ['beauty', 'mascara'],
 'brand': 'Essence',
 'sku': 'RCH45Q1A',
 'weight': 2,
 'dimensions': {'width': 23.17, 'height': 14.43, 'depth': 28.01},
 'warrantyInformation': '1 month warranty',
 'shippingInformation': 'Ships in 1 month',
 'availabilityStatus': 'Low Stock',
 'reviews': [{'rating': 2,
   'comment': 'Very unhappy with my purchase!',
   'date': '2024-05-23T08:56:21.618Z',
   'reviewerName': 'John Doe',
   'reviewerEmail': 'john.doe@x.dummyjson.com'},
  {'rating': 2,
   'comment': 'Not as described!',
   'date': '2024-05-23T08:56:21.618Z',
   'reviewerName': 'Nolan Gonzalez',
   'reviewerEmail': 'nolan.gonzalez@x.dummyjson.com'},

Iniciamos o agente como os anteriores.

In [123]:
prompt = hub.pull("hwchase17/react")

In [124]:
tools = [products, product]

In [125]:
agent = create_react_agent(
    chat_openai,
    tools,
    prompt)

In [126]:
agent_executor = AgentExecutor(
    agent=agent, tools=tools, verbose=True, handle_parsing_errors=True
)

Pelas descrições, o modelo LLM será capaz de realizar um "planejamento" sobre que ação tomar e quais dados precisa para cada ação.

In [127]:
agent_executor.invoke({"input": "Traga a lista de produtos da loja ShopAI."})



> Entering new AgentExecutor chain...
Preciso usar a função products para obter a lista de produtos da loja ShopAI.
Action: products
Action Input: [{'id': 1, 'title': 'Essence Mascara Lash Princess', 'description': 'The Essence Mascara Lash Princess is a popular mascara known for its volumizing and lengthening effects. Achieve dramatic lashes with this long-lasting and cruelty-free formula.', 'category': 'beauty', 'price': 9.99, 'discountPercentage': 7.17, 'rating': 4.94, 'stock': 5, 'tags': ['beauty', 'mascara'], 'brand': 'Essence', 'sku': 'RCH45Q1A', 'weight': 2, 'dimensions': {'width': 23.17, 'height': 14.43, 'depth': 28.01}, 'warrantyInformation': '1 month warranty', 'shippingInformation': 'Ships in 1 month', 'availabilityStatus': 'Low Stock', 'reviews': [{'rating': 2, 'comment': 'Very unhappy with my purchase!', 'date': '2024-05-23T08:56:21.618Z', 'reviewerName': 'John Doe', 'reviewerEmail': 'john.doe@x.dummyjson.com'}, {'rating': 2, 'comment': 'Not as described!', 'date': '2024

{'input': 'Traga a lista de produtos da loja ShopAI.',
 'output': "[{'title': 'Essence Mascara Lash Princess', 'price': 9.99}, {'title': 'Eyeshadow Palette with Mirror', 'price': 19.99}, {'title': 'Powder Canister', 'price': 14.99}]"}

In [128]:
agent_executor.invoke({"input": "Traga o produto com identificação 1 da loja ShopAI."})



> Entering new AgentExecutor chain...
Preciso usar a função product para encontrar o produto com a identificação 1.
Action: product
Action Input: "1"{'id': 1, 'title': 'Essence Mascara Lash Princess', 'description': 'The Essence Mascara Lash Princess is a popular mascara known for its volumizing and lengthening effects. Achieve dramatic lashes with this long-lasting and cruelty-free formula.', 'category': 'beauty', 'price': 9.99, 'discountPercentage': 7.17, 'rating': 4.94, 'stock': 5, 'tags': ['beauty', 'mascara'], 'brand': 'Essence', 'sku': 'RCH45Q1A', 'weight': 2, 'dimensions': {'width': 23.17, 'height': 14.43, 'depth': 28.01}, 'warrantyInformation': '1 month warranty', 'shippingInformation': 'Ships in 1 month', 'availabilityStatus': 'Low Stock', 'reviews': [{'rating': 2, 'comment': 'Very unhappy with my purchase!', 'date': '2024-05-23T08:56:21.618Z', 'reviewerName': 'John Doe', 'reviewerEmail': 'john.doe@x.dummyjson.com'}, {'rating': 2, 'comment': 'Not as described!', 'date': '202

{'input': 'Traga o produto com identificação 1 da loja ShopAI.',
 'output': "{'id': 1, 'title': 'Essence Mascara Lash Princess', 'description': 'The Essence Mascara Lash Princess is a popular mascara known for its volumizing and lengthening effects. Achieve dramatic lashes with this long-lasting and cruelty-free formula.', 'category': 'beauty', 'price': 9.99, 'discountPercentage': 7.17, 'rating': 4.94, 'stock': 5, 'tags': ['beauty', 'mascara'], 'brand': 'Essence', 'sku': 'RCH45Q1A', 'weight': 2, 'dimensions': {'width': 23.17, 'height': 14.43, 'depth': 28.01}, 'warrantyInformation': '1 month warranty', 'shippingInformation': 'Ships in 1 month', 'availabilityStatus': 'Low Stock', 'reviews': [{'rating': 2, 'comment': 'Very unhappy with my purchase!', 'date': '2024-05-23T08:56:21.618Z', 'reviewerName': 'John Doe', 'reviewerEmail': 'john.doe@x.dummyjson.com'}, {'rating': 2, 'comment': 'Not as described!', 'date': '2024-05-23T08:56:21.618Z', 'reviewerName': 'Nolan Gonzalez', 'reviewerEmail':